In [ ]:

!pip install crewai langchain openai langchain_community


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.5/155.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.1 MB/s eta 0:

In [ ]:
# ========== CHUNK 1: Setup and imports ==========

from crewai import Agent, Task, Crew
from langchain.chat_models import ChatOpenAI
import os
from google.colab import userdata

# Access the secret securely from Colab secrets
openai_api_key = userdata.get('OPENAI_API_KEY')

# You can then set it as an environment variable if needed, but it's often
# sufficient to just use the variable directly.
os.environ["OPENAI_API_KEY"] = openai_api_key

# Set your OpenAI API key (or load from .env)

# Initialize the LLM
llm = ChatOpenAI(
    model_name="gpt-4",  # or "gpt-4"
    temperature=0.3
)


/tmp/ipython-input-2681578742.py:18: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [ ]:
# Project Manager Agent
manager_agent = Agent(
    role='Project Manager',
    goal='Assign tasks to other agents based on the user query',
    backstory="You are an expert project manager who efficiently delegates tasks.",
    llm=llm,
    allow_delegation=True
)

# Research Specialist Agent
research_agent = Agent(
    role='Research Specialist',
    goal='Research the topic thoroughly and provide detailed insights',
    backstory="A highly skilled researcher who dives deep into any topic.",
    llm=llm
)

# Technical Writer Agent
writer_agent = Agent(
    role='Technical Writer',
    goal='Write a clear and concise report based on the research findings',
    backstory="An expert writer who crafts professional and understandable reports.",
    llm=llm
)

# Reviewer Agent
reviewer_agent = Agent(
    role='Quality Reviewer',
    goal='Review the final report and suggest improvements for accuracy and clarity',
    backstory="An experienced editor with an eye for detail and factual accuracy.",
    llm=llm
)


In [ ]:
def create_tasks(user_topic):
    task1 = Task(
        description=f"Research the following topic in depth: {user_topic}. Gather data, trends, and important points.",
        expected_output="A detailed research summary with references and examples.",
        agent=research_agent
    )

    task2 = Task(
        description="Based on the research findings, write a clear and concise report.",
        expected_output="A well-structured report summarizing the research findings.",
        agent=writer_agent
    )

    task3 = Task(
        description="Review the final report for accuracy, clarity, and improvements.",
        expected_output="A list of suggestions to improve the report.",
        agent=reviewer_agent
    )

    return [task1, task2, task3]


In [ ]:
# Main loop
while True:
    user_input = input("\n Enter your query (or type 'exit' to quit): ")

    if user_input.lower() in ['exit', 'quit']:
        print(" Exiting... Have a great day!")
        break

    # Step 1: Intent classification
    intent_prompt = (
        "You are an intelligent assistant that classifies the following query as either 'agent' or 'llm'. "
        "Respond only with 'agent' if the task requires step-by-step research, writing, and review. "
        "Respond only with 'llm' if the task is simple and can be handled directly by the language model.\n\n"
        f"Query: {user_input}\n\nAnswer:"
    )
    intent = llm.invoke(intent_prompt).content.strip().lower()

    if intent == 'agent':
        print("\n Starting Agent Tasks...\n")

        # Create tasks and crew
        tasks = create_tasks(user_input)
        crew = Crew(
            agents=[manager_agent, research_agent, writer_agent, reviewer_agent],
            tasks=tasks,
            verbose=False  #  No verbose output
        )

        # Run the crew and get the result
        result = crew.kickoff()

        # Print outputs task by task
        print("\n📄 Final Report Generated by Agents:\n")
        for idx, task in enumerate(tasks, start=1):
            print(f"\n Output from Task {idx} ({task.agent.role}):")
            print(task.output)
        print("\n📄 Final Consolidated Report:")
        print(result)

    elif intent == 'llm':
        print("\n Using LLM directly to answer your query...\n")
        response = llm.predict(user_input)
        print("\n Response:")
        print(response)

    else:
        print(" Couldn't understand the intent. Please try again.")


 Enter your query (or type 'exit' to quit): tell about NIIT LTD

 Using LLM directly to answer your query...



/tmp/ipython-input-2835039964.py:42: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict(user_input)



 Response:
NIIT Ltd is a global learning solutions company headquartered in Gurgaon, India. It was established in 1981 to provide information technology (IT) education and training. The company offers learning and knowledge solutions to individuals, enterprises, and institutions in information technology, business process outsourcing, executive management education, school education, communication and professional life skills, and vocational skills training.

NIIT's offerings include managed training services, a comprehensive suite of training outsourcing services that provide companies with a range of operational and strategic benefits. It also provides a range of learning solutions for corporations, including content development, training delivery and administration, technology and advisory services.

The company operates in three main segments: Corporate Learning Group (CLG), which offers managed training services; Skills and Careers Group (SNC), which offers a range of learning so

In [ ]:
exit()